## **Published research on promising Drugs**

This work search for recent research on drugs that shows improvement in human health against the current pandemic. It uses keywords and trial. The new keywords, drug names and concepts are published - slowly putting together a picture of drugs that may show promise for help fighting this pandemic. 


In [20]:
import spacy
import numpy as np
import pandas as pd
import re
import os
import json
from pprint import pprint
from copy import deepcopy
import math
import torch
!pip install rake-nltk
from rake_nltk import Rake
from nltk.corpus import stopwords
from rake_nltk import Metric, Rake

In [21]:
print ('python packages imported')

from IPython.core.display import display, HTML

def search_focus(df):
    dfa = df[df['abstract'].str.contains('covid')]
    dfb = df[df['abstract'].str.contains('-cov-2')]
    dfc = df[df['abstract'].str.contains('cov2')]
    dfd = df[df['abstract'].str.contains('ncov')]
    frames=[dfa,dfb,dfc,dfd]
    df = pd.concat(frames)
    df=df.drop_duplicates(subset='title', keep="first")
    return df

# load the meta data from the CSV file using 3 columns (abstract, title, authors),
df=pd.read_csv('metadata.csv', usecols=['title','journal','abstract','authors','doi','publish_time','sha','full_text_file'])
print ('total documents ',df.shape)
#drop duplicates
#df=df.drop_duplicates()
#drop NANs 
df=df.fillna('no data provided')
df = df.drop_duplicates(subset='title', keep="first")
# convert abstracts to lowercase
df["abstract"] = df["abstract"].str.lower()+df["title"].str.lower()
#show 5 lines of the new dataframe
df=search_focus(df)
df = df[df['publish_time'].str.contains('2020')]
print ('COVID-19 focused documents ',df.shape)
df.head()

# add full text back after testing

python packages imported
total documents  (57366, 8)
COVID-19 focused documents  (5615, 8)


,sha,title,doi,abstract,publish_time,authors,journal,full_text_file
12519,b5d303cbcfe6be92d733ec593118b388db77452e,Complete Genome Sequence of a 2019 Novel Coron...,10.1128/mra.00169-20,a complete genome sequence was obtained for a ...,2020-03-12,"Sah, Ranjit; Rodriguez-Morales, Alfonso J.; Jh...",Microbiol Resour Announc,comm_use_subset
12520,ce358c18aac69fc83c7b2e9a7dca4a43b0f60e2e,First cases of coronavirus disease 2019 (COVID...,10.2807/1560-7917.es.2020.25.9.2000178,"in the who european region, covid-19 surveilla...",2020-03-05,"Spiteri, Gianfranco; Fielding, James; Diercke,...",Euro Surveill,comm_use_subset
12525,a1f9e6d1c2717f4555275f25d6169063b7f8bb23,What further should be done to control COVID-1...,10.1186/s12916-020-01551-8,no data providedwhat further should be done to...,2020-03-13,"He, Zhenjian",BMC Med,comm_use_subset
12536,no data provided,"Is A Problem Shared, A Problem Halved? Not Alw...",10.5005/jp-journals-10071-23365,"how to cite this article: bhosale s, kulkarni ...",2020-02-23,"Bhosale, Shilpushp; Kulkarni, Atul P",Indian J Crit Care Med,no data provided
12537,4b29b65801480de25dd4b503bfd7939a1884b97e,The Effects of Social Support on Sleep Quality...,10.12659/msm.923549,background: coronavirus disease 2019 (covid-19...,2020-03-05,"Xiao, Han; Zhang, Yan; Kong, Desheng; Li, Shiy...",Med Sci Monit,noncomm_use_subset


In [22]:
import functools
def search_focus_shape(df,focus):
    df1=df[functools.reduce(lambda a, b: a&b, (df['abstract'].str.contains(s) for s in focus))]
    return df1

focus_term='drug'
df1=search_focus_shape(df,focus_term)
print ('focus term: ',focus_term)
print ('# focused papers',df1.shape)

focus term:  drug
# focused papers (4896, 8)


In [23]:
r = Rake(ranking_metric=Metric.DEGREE_TO_FREQUENCY_RATIO,min_length=2, max_length=2) # Uses stopwords for english from NLTK, and all puntuation characters.Please note that "hello" is not included in the list of stopwords.

def extract_data(text,word):
    extract=''
    if word in text:
        #text = re.sub(r'[^\w\s]','',text)
        res = [i.start() for i in re.finditer(word, text)]
        after=text[res[0]:res[0]+15]
        before=text[res[0]-15:res[0]]
        raw = before+after
        parts=raw.split()
        parts = parts[1:-1]
        extract= ' '.join(parts)
        extract=extract.replace('drugs','drug')
    return extract
text=''
for index, row in df1.iterrows():
    extracted=extract_data(row['abstract'],focus_term)
    if extracted!='':
        text=text+' '+extracted
a=r.extract_keywords_from_text(text)
term_list=r.get_ranked_phrases()
term_list = sorted(term_list, key=str.lower)
#c=r.get_ranked_phrases_with_scores()
print(term_list)
#print(c)
print('___________________')

['19 drug', '2 logs', '3 ).', '3d number', 'administered drug', 'anticytokine drug', 'antimalarial drug', 'antitumor drug', 'antiviral drug', 'antivirus drug', 'approved drug', 'arb drug', 'available drug', 'candidate drug', 'clinical drug', 'cmt drug', 'data provideddrug', 'designed drug', 'driven drug', 'drug ),', 'drug azt', 'drug center', 'drug detection', 'drug drug', 'drug effects', 'drug labeling', 'drug therapy', 'drug type', 'drug use', 'drug used', 'drug week', 'effective drug', 'efficacious drug', 'existing drug', 'find drug', 'flu drug', 'gold drug', 'hcv drug', 'hiv drug', 'including drug', 'induced attention', 'inhalable drug', 'inhibitory drug', 'narcotic drug', 'new drug', 'novel drug', 'potential drug', 'promising drug', 'proven drug', 'purchasable drug', 'repurposed drug', 'several drug', 'specific drug', 'structural drug', 'therapeutic drug', 'three drug', 'treat anti', 'treat resource', 'treat target', 'unlicensed drug', 'us food', 'viral drug', 'virus drug']
______

In [24]:
# custom sentence score
def score_sentence_prob(search,sentence,focus):
    final_score=0
    keywords=search.split()
    sent_parts=sentence.split()
    word_match=0
    missing=0
    for word in keywords:
        word_count=sent_parts.count(word)
        word_match=word_match+word_count
        if word_count==0:
            missing=missing+1
    percent = 1-(missing/len(keywords))
    final_score=abs((word_match/len(sent_parts)) * percent)
    if missing==0:
        final_score=final_score+.05
    if focus in sentence:
        final_score=final_score+.05
    return final_score

def score_docs(df,focus,search):
    df_results = pd.DataFrame(columns=['date','study','link','excerpt','score'])
    df1=df[functools.reduce(lambda a, b: a&b, (df['abstract'].str.contains(s) for s in focus))]
    master_text=''
    for index, row in df1.iterrows():
        pub_sentence=''
        sentences=row['abstract'].split('.')
        hi_score=0
        for sentence in sentences:
            if len(sentence)>75 and search in sentence:
                rel_score=score_sentence_prob(search,sentence,focus)
                #rel_score=score_sentence(search,sentence)
                if rel_score>.05:
                    #print (sentence,rel_score)
                    pub_sentence=pub_sentence+' '+sentence+' '+str(round(rel_score,2))
                    if rel_score>hi_score:
                        hi_score=rel_score
                    master_text=master_text+' '+pub_sentence
        if pub_sentence!='':
            #print (row['abstract'])
            #print ('------------------')
            link=row['doi']
            linka='https://doi.org/'+link
            to_append = [row['publish_time'],row['title'],linka,pub_sentence,hi_score]
            df_length = len(df_results)
            df_results.loc[df_length] = to_append
    df_results=df_results.sort_values(by=['date'], ascending=False)

    return df_results
for term in term_list:
    if 'drug' in term and any(map(str.isdigit, term))==False and ')' not in term:
        df_results=score_docs(df,focus_term,term)
        if df_results.empty==False:
            print (term)
            df_table_show=HTML(df_results.to_html(escape=False,index=False))
            display(df_table_show)

administered drug


date,study,link,excerpt,score
2020-03-30,"Medical treatment of 55 patients with COVID-19 from seven cities in northeast China who fully recovered: a single-center, retrospective, observational study",https://doi.org/10.1101/2020.03.28.20045955,"administered drugs and intervention times were compared in 47 and eight patients with mild and severe symptoms, respectively 0.08",0.077778


anticytokine drug


date,study,link,excerpt,score
2020-04-04,"Management of Patients on Dialysis and With Kidney Transplantation During the SARS-CoV-2 (COVID-19) Pandemic in Brescia, Italy",https://doi.org/10.1016/j.ekir.2020.04.001,"in the second inflammatory phase of the disease, immunosuppressive strategies (for example with glucocorticoids or anticytokine drugs) may be considered 0.08",0.075


antimalarial drug


...

date,study,link,excerpt,score
2020-03-27,Modelling SARS-CoV-2 Dynamics: Implications for Therapy,https://doi.org/10.1101/2020.03.23.20040493,"furthermore, we used our viral dynamics model to predict the effectiveness of unlicensed drugs that have different methods of action 0.08",0.075


viral drug


date,study,link,excerpt,score
2020-06-30,Current knowledge about the antivirals remdesivir (GS-5734) and GS-441524 as therapeutic options for coronaviruses,https://doi.org/10.1016/j.onehlt.2020.100128,"remdesivir (gs-5734™) is a broad-spectrum antiviral drug that is now being tested as a potential treatment for covid-19 in international, multi-site clinical trials 0.07",0.071739
2020-04-21,Elucidating the differences in the molecular mechanism of receptor binding between 2019-nCoV and the SARS-CoV viruses using computational tools,https://doi.org/10.1101/2020.04.21.053009,many biotechnology laboratories are rushing to develop therapeutic antibodies and antiviral drugs for treatment of this viral disease 0.08,0.077778
2020-04-21,Human iPSC-Derived Cardiomyocytes are Susceptible to SARS-CoV-2 Infection,https://doi.org/10.1101/2020.04.21.051912,"these studies show that sars-cov-2 can infect hipsc-cms in vitro, establishing a model for elucidating the mechanisms of infection and potentially a cardiac-specific antiviral drug screening platform 0.07",0.068519
2020-04-19,Recent progress and challenges in drug development against COVID-19 coronavirus (SARS-CoV-2) - an update on the status,https://doi.org/10.1016/j.meegid.2020.104327,favilavir is an antiviral drug that is approved in japan for common influenza treatment and is now approved to treat symptoms of covid-19 in china 0.07,0.070000
2020-04-19,Déjà vu: Stimulating open drug discovery for SARS-CoV-2,https://doi.org/10.1016/j.drudis.2020.03.019,"here, we propose, based on open antiviral drug discovery approaches for previous outbreaks, that there could still be gaps in our approach to drug discovery 0.09",0.090000
2020-04-15,TMPRSS2 and furin are both essential for proteolytic activation and spread of SARS-CoV-2 in human airway epithelial cells and provide promising drug targets,https://doi.org/10.1101/2020.04.15.042085,"currently, there is no approved antiviral drug for treating covid-19 patients and there is an urgent need for specific antiviral therapies and vaccines 0.07",0.071739
2020-04-15,Epidemiological investigation and intergenerational clinical characteristics of 24 COVID-19 patients associated with supermarket cluster,https://doi.org/10.1101/2020.04.11.20058891,"treatment: antiviral drug (100%), chinese medicine (100%), common oxygen therapy (45 0.1",0.095455
2020-04-14,Calcium channel blocker amlodipine besylate is associated with reduced case fatality rate of COVID-19 patients with hypertension,https://doi.org/10.1101/2020.04.08.20047134,development of effective anti-viral drug is in urgent need to treat covid-19 patients 0.09,0.088462
2020-04-14,Efficacy of remdesivir versus placebo for the treatment of COVID-19: A protocol for systematic review and meta-analysis of randomized controlled trials,https://doi.org/10.1101/2020.04.09.20059196,"remdesivir, a nucleotide-analogue antiviral drug developed for ebola, is determined to prevent and stop infections with covid-19, while results are yet controversial 0.07",0.072727
2020-04-14,Dos terapias conocidas podrían ser efectivas como adyuvantes en el paciente crítico infectado por COVID-19,https://doi.org/10.1016/j.redar.2020.03.004,"unfortunately, there is not yet a specific vaccine or effective antiviral drug for treating covid-19 0.08",0.083333


virus drug


date,study,link,excerpt,score
2020,Pharmacotherapeutics for the New Coronavirus Pneumonia,https://doi.org/10.3760/cma.j.issn.1001-0939.2020.0012,"combination of ribavirin and interferon-α is recommended by the 5(th) edition national health commission's regimen (revised edition) because of the effect on mers (middle east respiratory syndrome), and the effectiveness of lopinavir/ritonavir and remdisivir needs to be confirmed by randomized controlled trial (rct), given the situation of no specific antivirus drug on ncp is unavailable 0.06",0.059091
